In [1]:
import datetime
import time as time_module
import sys
import os 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import theano
import matplotlib
import pymc3 as pm
import theano.tensor as tt

try: 
    import covid19_inference as cov19
except ModuleNotFoundError:
    sys.path.append('..')
    import covid19_inference as cov19



First we load the desired data from a source we also need to tell the class to download the data

In [2]:
rki = cov19.data_retrieval.Rki()
rki.download_all_available_data()

../covid19_inference/data_retrieval.py:478: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df = pd.concat([df, df_temp], ignore_index=True)


Finished downloading


,Altersgruppe,AnzahlFall,AnzahlGenesen,AnzahlTodesfall,Bundesland,Geschlecht,Landkreis,Meldedatum,NeuGenesen,NeuerFall,Refdatum,date,date_ref
0,A00-A04,1,1,0,Rheinland-Pfalz,W,SK Worms,1585612800000,0,0,1584316800000,2020-03-31 02:00:00,2020-03-16 01:00:00
1,A05-A14,1,0,0,Rheinland-Pfalz,M,SK Worms,1586304000000,-9,0,1586304000000,2020-04-08 02:00:00,2020-04-08 02:00:00
2,A05-A14,1,1,0,Rheinland-Pfalz,M,SK Worms,1586476800000,0,0,1585872000000,2020-04-10 02:00:00,2020-04-03 02:00:00
3,A05-A14,1,0,0,Rheinland-Pfalz,M,SK Worms,1586649600000,-9,0,1586476800000,2020-04-12 02:00:00,2020-04-10 02:00:00
4,A15-A34,1,1,0,Rheinland-Pfalz,M,SK Worms,1584403200000,0,0,1583971200000,2020-03-17 01:00:00,2020-03-12 01:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
103996,A80+,1,0,0,Bayern,W,SK Ingolstadt,1586736000000,-9,0,1586476800000,2020-04-13 02:00:00,2020-04-10 02:00:00
103997,A80+,1,0,1,Bayern,W,SK Ingolstadt,1586736000000,-9,0,1586563200000,2020-04-13 02:00:00,2020-04-11 02:00:00
103998,A80+,2,0,0,Bayern,W,SK Ingolstadt,1586736000000,-9,0,1586736000000,2020-04-13 02:00:00,2020-04-13 02:00:00
103999,A80+,2,0,0,Bayern,W,SK Ingolstadt,1587081600000,-9,1,1586995200000,2020-04-17 02:00:00,2020-04-16 02:00:00


Wait for the download to finish. It will print a message!
We can now access this downloaded data by the attribute
```
rki.data
```

In [3]:
df_bundeslaender = rki.filter_all_bundesland(rki.data, '2020-03-10', '2020-04-13')
new_cases_obs = np.diff(np.array(df_bundeslaender),axis=0)[:,:]


date_begin_data = datetime.datetime(2020,3,10)
date_end_data   = datetime.datetime(2020,3,13)
diff_data_sim = 16 # should be significantly larger than the expected delay, in 
                   # order to always fit the same number of data points.
num_days_forecast = 10

prior_date_mild_dist_begin =  datetime.datetime(2020,3,9)
prior_date_strong_dist_begin =  datetime.datetime(2020,3,16)
prior_date_contact_ban_begin =  datetime.datetime(2020,3,23)

change_points = [dict(pr_mean_date_transient = prior_date_mild_dist_begin,
                      pr_sigma_date_transient = 6,
                      pr_median_lambda = 0.2,
                      pr_sigma_lambda = 1),
                 dict(pr_mean_date_transient = prior_date_strong_dist_begin,
                      pr_sigma_date_transient = 6,
                      pr_median_lambda = 1/8,
                      pr_sigma_lambda = 1),
                 dict(pr_mean_date_transient = prior_date_contact_ban_begin,
                      pr_sigma_date_transient = 6,
                      pr_median_lambda = 1/8/2,
                      pr_sigma_lambda = 1)]

In [4]:
params_model = dict(new_cases_obs = new_cases_obs,
                    date_begin_data = date_begin_data,
                    num_days_forecast = num_days_forecast,
                    diff_data_sim = diff_data_sim,
                    N_population = 83e6) 
# normally one would put as N_population an array with the number of inhabitants of 
# of each state

with cov19.Cov19_Model(**params_model) as model:
    lambda_t_log = cov19.lambda_t_with_sigmoids(pr_median_lambda_0 = 0.4,
                                                change_points_list = change_points)
    
    new_I_t = cov19.SIR(lambda_t_log, pr_median_mu=1/8)
    
    new_cases_inferred_raw = cov19.delay_cases(new_I_t, pr_median_delay=10, 
                                               pr_median_scale_delay=0.3)
    
    #new_cases_inferred = cov19.week_modulation(new_cases_inferred_raw)
    
    cov19.student_t_likelihood(new_cases_inferred_raw)


pr_median_transient_len was set to default value 4
pr_sigma_transient_len was set to default value 1
pr_median_transient_len was set to default value 4
pr_sigma_transient_len was set to default value 1
pr_median_transient_len was set to default value 4
pr_sigma_transient_len was set to default value 1


WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


In [ ]:
trace = pm.sample(model=model, tune=500, draws=500)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma_obs, delay_L2_raw, delay_L1, sigma_delay_L2, I_begin, mu, transient_len_3_L2_raw, transient_len_3_L1, sigma_transient_len_3_L2, transient_len_2_L2_raw, transient_len_2_L1, sigma_transient_len_2_L2, transient_len_1_L2_raw, transient_len_1_L1, sigma_transient_len_1_L2, transient_day_3_L2_raw, transient_day_3_L1, sigma_transient_day_3_L2, transient_day_2_L2_raw, transient_day_2_L1, sigma_transient_day_2_L2, transient_day_1_L2_raw, transient_day_1_L1, sigma_transient_day_1_L2, lambda_3_L2_raw, lambda_3_L1, sigma_lambda_3_L2, lambda_2_L2_raw, lambda_2_L1, sigma_lambda_2_L2, lambda_1_L2_raw, lambda_1_L1, sigma_lambda_1_L2, lambda_0_L2_raw, lambda_0_L1, sigma_lambda_0_L2]
Sampling 4 chains, 9 divergences:  80%|████████  | 3210/4000 [24:10<10:06,  1.30draws/s] 

In [14]:
plt.plot(np.median(trace.new_cases, axis=0))

AttributeError: 'MultiTrace' object has no attribute 'new_cases'